In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import sys
import QuantLib as ql
sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python")
sys.path.insert(0, "/home/christophe/gitlab/norm/release/api/python/ct")
sys.path.insert(0, "/home/christophe/gitlab/norm/test/scripts/python/Unitaire")

import import_allocation as ia
import import_pricing as pr
import TestWREmodelingCovFiltering as cf
#import openaml_importer.model as models

In [2]:
with open("data/assets.json", "r") as f:
    data = json.load(f) 

In [3]:
type(data)

dict

In [4]:
dd = data['assets']

In [5]:
# list of swaps ESTR
#pd.set_option('display.max_rows', 300)
col=['maturityDate', 'currency', 'quotationdate', 'quotationvalue', 'side0', 'cur0', 'ratetype', 'frequency','index','marginAdd', 'marginMult', 'side1','cur1', 'ratetype1','frequency', 'rate'] #
pdswap = pd.DataFrame(columns=col)
j=0
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("_ESTR" in dd[i]['name'])):
        #print('tt',dd[i]['legs'][0]['interest'].get('index'))
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['ccy'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['ccy'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][0]['interest']['marginAdd'], 
                             dd[i]['legs'][0]['interest']['marginMult'], 
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['ccy'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate')
                            ]], 
                             columns=col, index=[dd[i]['name']])
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
display(pdswap)

/tmp/ipykernel_414155/454589895.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,currency,quotationdate,quotationvalue,side0,cur0,ratetype,frequency,index,marginAdd,marginMult,side1,cur1,ratetype1,frequency,rate
SW20250131_ESTR O/3.246,2025-01-31,EUR,2024-12-19,100.378497,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,3.2460
SW20250113_ESTR O/2.9599,2025-01-13,EUR,2024-12-19,100.000814,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,2.9599
SW20250908_ESTR O/2.813,2025-09-08,EUR,2024-12-19,99.786749,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,2.8130
SW20250131_ESTR O/3.246,2025-01-31,EUR,2024-12-19,100.378497,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,3.2460
SW20250205_ESTR O/3.0175,2025-02-05,EUR,2024-12-19,100.001246,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,3.0175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SW20251112_ESTR O/2.3055,2025-11-12,EUR,2024-12-19,100.006444,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,2.3055
SW20250207_ESTR O/3.0094,2025-02-07,EUR,2024-12-19,100.002011,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,3.0094
SW20250718_ESTR O/3.23,2025-07-18,EUR,2024-12-19,99.631221,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,3.2300
SW20250219_ESTR O/2.9164,2025-02-19,EUR,2024-12-19,100.011090,RECEIVE,EUR,FLOATING,INFI,6741831,0.0,1.0,PAY,EUR,FIXED,INFI,2.9164


In [6]:
# list of swaps SOFR
j=0
pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['ccy'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['ccy'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][0]['interest']['marginAdd'], 
                             dd[i]['legs'][0]['interest']['marginMult'], 
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['ccy'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate')
                            ]], 
                             columns=col, index=[dd[i]['name']])
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

15


/tmp/ipykernel_414155/260048983.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,currency,quotationdate,quotationvalue,side0,cur0,ratetype,frequency,index,marginAdd,marginMult,side1,cur1,ratetype1,frequency,rate
SW20251113_SOFR O/4.2587,2025-11-13,USD,2024-12-19,100.016363,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.2587
SW20250211_SOFR O/4.46,2025-02-11,USD,2024-12-19,100.060533,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.4600
SW20250407_SOFR O/4.5635,2025-04-07,USD,2024-12-19,100.027895,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.5635
SW20250606_SOFR O/4.391,2025-06-06,USD,2024-12-19,100.102703,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.3910
SW20250508_SOFR O/4.411,2025-05-08,USD,2024-12-19,99.965975,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.4110
SW20251114_SOFR O/4.219,2025-11-14,USD,2024-12-19,100.033389,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.2190
SW20250626_SOFR O/5.0435,2025-06-26,USD,2024-12-19,99.684772,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,5.0435
SW20250310_SOFR O/4.6098,2025-03-10,USD,2024-12-19,100.018657,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.6098
SW20251002_SOFR O/3.834,2025-10-02,USD,2024-12-19,100.523361,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,3.8340
SW20250718_SOFR O/4.0145,2025-07-18,USD,2024-12-19,100.366475,RECEIVE,USD,FLOATING,ANNU,5967593,0.0,1.0,PAY,USD,FIXED,ANNU,4.0145


In [7]:
# list of swaps no SOFR and ESTR and EU
j=0
col=['maturityDate', 'currency', 'quotationdate', 'quotationvalue', 'side0', 'cur0', 'ratetype', 'frequency',  'rate', 'side1', 'cur1', 'ratetype1','frequency',  'index','marginAdd', 'marginMult' ] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" not in dd[i]['name']) and ("_ESTR" not in dd[i]['name'])and ("EUR" not in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['ccy'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['ccy'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             #dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][0]['interest'].get('rate'),
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['ccy'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             dd[i]['legs'][1]['interest']['index']['_id'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest']['marginAdd'], 
                             dd[i]['legs'][1]['interest']['marginMult']
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

2


/tmp/ipykernel_414155/2045291089.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,currency,quotationdate,quotationvalue,side0,cur0,ratetype,frequency,rate,side1,cur1,ratetype1,frequency,index,marginAdd,marginMult
SW20260126_2.172/ESTR O,2026-01-24,EUR,2024-12-19,100.114127,RECEIVE,EUR,FIXED,ANNU,2.172,PAY,EUR,FLOATING,ANNU,6741831,0.0,1.0
SW20280325_5.75/HUF BU,2028-03-25,HUF,2024-12-19,98.812563,RECEIVE,HUF,FIXED,ANNU,5.750,PAY,HUF,FLOATING,SEME,549433,0.0,1.0


In [8]:
# list of swaps no SOFR and ESTR and EU
j=0
col=['maturityDate', 'currency', 'quotationdate', 'quotationvalue', 'side0', 'currency0', 'ratetype', 'frequency', 'index' ,'marginAdd', 'marginMult' ,'side1','currency1', 'ratetype1','frequency','rate'] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['assetType']=='SWAP_ON_RATE') and ("SOFR" not in dd[i]['name']) and ("_ESTR" not in dd[i]['name'])and ("EUR"  in dd[i]['name'])):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['maturityDate'],
                             dd[i]['ccy'],
                             dd[i]['quotations'][0]['date'],
                             dd[i]['quotations'][0]['close'],
                             dd[i]['legs'][0]['side'], 
                             dd[i]['legs'][0]['ccy'], 
                             dd[i]['legs'][0]['interest']['rateType'], 
                             dd[i]['legs'][0]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][0]['interest']['dayCountMethod'], 
                             dd[i]['legs'][0]['interest']['index']['_id'], 
                             dd[i]['legs'][0]['interest']['marginAdd'], 
                             dd[i]['legs'][0]['interest']['marginMult'],
                             dd[i]['legs'][1]['side'], 
                             dd[i]['legs'][1]['ccy'], 
                             dd[i]['legs'][1]['interest']['rateType'], 
                             dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             dd[i]['legs'][1]['interest'].get('rate')
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
print(j)
pdswap

2


/tmp/ipykernel_414155/2598958411.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,maturityDate,currency,quotationdate,quotationvalue,side0,currency0,ratetype,frequency,index,marginAdd,marginMult,side1,currency1,ratetype1,frequency,rate
SW20290619_EUR EU/2.8187,2029-06-19,EUR,2024-12-19,96.071931,RECEIVE,EUR,FLOATING,SEME,251377,0.0,1.0,PAY,EUR,FIXED,ANNU,2.8187
SW20290805_EUR EU/2.5602,2029-08-05,EUR,2024-12-19,99.030237,RECEIVE,EUR,FLOATING,SEME,251377,0.0,1.0,PAY,EUR,FIXED,ANNU,2.5602


In [9]:
# index
j=0
col=['index' , 'assetType', 'dates', 'last', 'currency'] 
#col=['maturityDate', 'side0', 'ratetype', 'frequency', 'daycountmethod' ,'index','ratetype1','frequency', 'daycountmethod', 'rate'] #

pdswap = pd.DataFrame(columns=col)
for i in range(len(dd)):
    if ((dd[i]['_id']=='251377')or (dd[i]['_id']=='6741831')or (dd[i]['_id']=='549433')or (dd[i]['_id']=='5967593')or (dd[i]['_id']=='6741831')):
        j=j+1
        pdnew=pd.DataFrame([[dd[i]['_id'],
                             dd[i]['assetType'], 
                             dd[i]['quotations'][0]['date'], 
                             dd[i]['quotations'][0]['last'], 
                             dd[i]['ccy']
                             #dd[i]['legs'][0]['interest']['index']['_id'], 
                             #dd[i]['legs'][1]['side'], 
                             #dd[i]['legs'][1]['interest']['rateType'], 
                             #dd[i]['legs'][1]['interest']['paymentFrequency']['code'], 
                             #dd[i]['legs'][1]['interest']['dayCountMethod'],
                             #dd[i]['legs'][1]['interest'].get('rate'),
                            ]], 
                             columns=col, index=[dd[i]['name']])        
        pdswap = pd.concat([pdswap, pdnew], ignore_index=False)
        #print(dd[i])
print(j)
pdswap

4


/tmp/ipykernel_414155/2197306129.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pdswap = pd.concat([pdswap, pdnew], ignore_index=False)


,index,assetType,dates,last,currency
EURIBOR 6M,251377,MONEY_MARKET_INDEX,2024-12-19,2.637000,EUR
HUF 6 MOIS BUBOR,549433,MONEY_MARKET_INDEX,2024-12-19,6.480000,HUF
SOFR OVERNIGHT RATE,5967593,MONEY_MARKET_INDEX,2024-12-18,4.570000,USD
ESTR OIS,6741831,MONEY_MARKET_INDEX_OIS,2024-12-20,3.739111,EUR


# Valo du SWAP SW20260126_2.172/ESTR O

In [1]:
#quotation 100.114, 2024-12-19

In [11]:
euribordates= np.array([7.,21.,63.,126.,252.])/252.0
euriborvaleurs= np.array([2.895,2.795,2.839,2.637,2.469])
euribordates= np.array([7.,21.,63.,126.,252., 504, 756, 1008, 1260])/252.0
euriborvaleurs= np.array([2.895,2.795,2.839,2.637,2.469, 2.2, 2.3, 2.35, 2.4])
euribordates= np.array([7.,21.,63.,126.,252., 504, 756, 1008])/252.0
euriborvaleurs= np.array([2.895,2.795,2.839,2.637, 2.375	, 2.181, 2.154, 2.156])



ncurve = euribordates.shape[0]

In [12]:
pricing = pr.Pricing(pathlib='/home/christophe/gitlab/norm/release/lib/linux64/')

Loading Pricing module:
Pricing module loaded.


In [13]:
#19/12/2024
#2 PAIEMENTS
#24/1/2025  ---> 
#24/1/2026
m=2
t1=np.array([22., 252.+22.])/252.0
t2=np.array([23., 252.+23.])/252.0
f12=np.zeros(m)
info = np.zeros(1,dtype='int64')

In [14]:
f12

array([0., 0.])

In [15]:
# coupon rate
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, euribordates, euriborvaleurs, m, t1, t2, interpmethod, f12, info )

 r1=   2.7960476190476187        2.3580634920634918     
 r2=   2.7970952380952383        2.3572936507936508     
 f12=   2.8201428571428657        2.1463571428571897     


In [16]:
# actualization rate
t1=np.array([0., 0.])/252.0
t2=np.array([23., 252.+23.])/252.0
actrat=np.zeros(m)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, euribordates, euriborvaleurs, m, t1, t2, interpmethod, actrat, info )
actrat

 r1=   2.8950000000000000        2.8950000000000000     
 r2=   2.7970952380952383        2.3572936507936508     
 f12=   2.7970952380952383        2.3572936507936508     


array([2.79709524, 2.35729365])

In [17]:
# jambe flottante
ndatesFL = m
datesFL = t2
actualizationratesFL = actrat/100.0
principalFL = 1.0
yearfractionFL = 1.0/250
floatingrate = f12/100.0

# jambe fixe
ndatesFIX = m
datesFIX = t2
actualizationratesFIX = actrat/100.0
principalFIX = 1.0
yearfractionFIX = 1.0/250
fixedrate = 2.172/100.0
buyer=1

#valo
fixedleg = np.zeros(1)
floatingleg = np.zeros(1)
price = np.zeros(1)
pricing.WREpricingIRS2( ndatesFL,  datesFL,  actualizationratesFL,   principalFL,  yearfractionFL, floatingrate,
                      ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, 
                      fixedrate, buyer, fixedleg, floatingleg, price, info)


 principal actualized from fixed leg=  0.97460361956532982     
   1.0000000000000000        2.3572936507936508E-002   1.0912698412698412     
 fixedleg=   1.7133204900660537E-004
 principal actualized from floating leg=  0.97460361956532982     
 test   1.0000000000000000        2.3572936507936508E-002   1.0912698412698412                2
 floatingleg=   1.9619199697841895E-004
 fixedleg=   1.7133204900660537E-004
 floatingleg=   1.9619199697841895E-004
 P=  -2.4859947971813583E-005


In [18]:
quote=floatingleg[0]/fixedleg[0]
quote

np.float64(1.145098060263408)

# Valo du SWAP SW20290619_EUR EU/2.8187

In [19]:
#  SW20290619_EUR EU/2.8187¶  mat :2029-06-19
#19/12/2024
# floating, SEMI
# fixe ANNU : 2025, 2026, 2027, 2028, 2029-06-19
# 5 PAIEMENTS
#24/1/2025  ---> 
#24/1/2026
# quotation 96.07

# pate fixe :
m=5
deca=126.0
year=252.0
# t1=np.array([0,year,2*year, 3*year, 4*year])/252.0
t1=np.array([0,0,0, 0, 0.0])/252.0
t2=np.array([deca,deca+year,deca+2*year, deca+3*year, deca+4*year])/year
f12=np.zeros(m)
info = np.zeros(1,dtype='int64')
actratfixe=np.zeros(m)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, euribordates, euriborvaleurs, m, t1, t2, interpmethod, actratfixe, info )

 r1=   2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000     
 r2=   2.6370000000000000        2.2780000000000000        2.1675000000000000        2.1550000000000002        2.1560000000000001     
 f12=   2.6370000000000000        2.2780000000000000        2.1675000000000000        2.1550000000000002        2.1560000000000001     


In [20]:
actrat

array([2.79709524, 2.35729365])

In [21]:
# pate flottante actratflot :
mfl=9
deca=126.0
semi=126.0
# t1=np.array([0,year,2*year, 3*year, 4*year])/252.0
t1fl=np.zeros(mfl)/year
t2fl=np.array([deca,deca+semi,deca+2*semi, deca+3*semi, deca+4*semi, deca+5*semi, deca+6*semi, deca+7*semi, deca+8*semi])/year
info = np.zeros(1,dtype='int64')
actratflot=np.zeros(mfl)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, euribordates, euriborvaleurs, mfl, t1fl, t2fl, interpmethod, actratflot, info )

 r1=   2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000        2.8950000000000000     
 r2=   2.6370000000000000        2.3750000000000000        2.2780000000000000        2.1810000000000000        2.1675000000000000        2.1539999999999999        2.1550000000000002        2.1560000000000001        2.1560000000000001     
 f12=   2.6370000000000000        2.3750000000000000        2.2780000000000000        2.1810000000000000        2.1675000000000000        2.1539999999999999        2.1550000000000002        2.1560000000000001        2.1560000000000001     


In [22]:
actratflot

array([2.637 , 2.375 , 2.278 , 2.181 , 2.1675, 2.154 , 2.155 , 2.156 ,
       2.156 ])

In [23]:
# pate flottante floatrate :
deca=126.0
semi=126.0
t1fl=np.array([0,0+semi,0+2*semi, 0+3*semi, 0+4*semi, 0+5*semi, 0+6*semi, 0+7*semi, 0+8*semi])/252.0
t2fl=np.array([deca,deca+semi,deca+2*semi, deca+3*semi, deca+4*semi, deca+5*semi, deca+6*semi, deca+7*semi, deca+8*semi])/year
info = np.zeros(1,dtype='int64')
floatrate=np.zeros(mfl)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, euribordates, euriborvaleurs, mfl, t1fl, t2fl, interpmethod, floatrate, info )

 r1=   2.8950000000000000        2.6370000000000000        2.3750000000000000        2.2780000000000000        2.1810000000000000        2.1675000000000000        2.1539999999999999        2.1550000000000002        2.1560000000000001     
 r2=   2.6370000000000000        2.3750000000000000        2.2780000000000000        2.1810000000000000        2.1675000000000000        2.1539999999999999        2.1550000000000002        2.1560000000000001        2.1560000000000001     
 f12=   2.6370000000000000        2.1130000000000000        2.0839999999999996        1.8900000000000006        2.1135000000000002        2.0864999999999991        2.1610000000000014        2.1630000000000003        2.1559999999999988     


In [24]:
floatrate

array([2.637 , 2.113 , 2.084 , 1.89  , 2.1135, 2.0865, 2.161 , 2.163 ,
       2.156 ])

In [25]:
t2

array([0.5, 1.5, 2.5, 3.5, 4.5])

In [26]:
actratfixe

array([2.637 , 2.278 , 2.1675, 2.155 , 2.156 ])

In [27]:
# jambe flottante
ndatesFL = mfl
datesFL = t2fl
actualizationratesFL = actratflot/100.0
principalFL = 1.0
yearfractionFL = 1.0#/2.0
floatingrate = floatrate/100.0

# jambe fixe
ndatesFIX = m
datesFIX = t2
actualizationratesFIX = actratfixe/100.0
principalFIX = 1.0
yearfractionFIX = 1.0
fixedrate = 2.8187/100.0
buyer=1

#valo
fixedleg = np.zeros(1)
floatingleg = np.zeros(1)
price = np.zeros(1)
pricing.WREpricingIRS2( ndatesFL,  datesFL,  actualizationratesFL,   principalFL,  yearfractionFL, floatingrate,
                      ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, 
                      fixedrate, buyer, fixedleg, floatingleg, price, info)

 principal actualized from fixed leg=  0.90753785519466001     
   1.0000000000000000        2.1560000000000003E-002   4.5000000000000000     
 fixedleg=  0.13347813920557094     
 principal actualized from floating leg=  0.90753785519466001     
 test   1.0000000000000000        2.1560000000000003E-002   4.5000000000000000                9
 floatingleg=  0.18392080023676252     
 fixedleg=  0.13347813920557094     
 floatingleg=  0.18392080023676252     
 P=  -5.0442661031191577E-002


In [28]:
quote=floatingleg[0]*100/fixedleg[0]
quote

np.float64(137.79095313390934)

In [29]:
actualizationratesFL

array([0.02637 , 0.02375 , 0.02278 , 0.02181 , 0.021675, 0.02154 ,
       0.02155 , 0.02156 , 0.02156 ])

In [30]:
actratflot

array([2.637 , 2.375 , 2.278 , 2.181 , 2.1675, 2.154 , 2.155 , 2.156 ,
       2.156 ])

In [31]:
# priceIRS

# jambe flottante
ndatesFL = 1
datesFL = t2fl[0]
actualizationratesFL = actratflot[0]/100.0
principalFL = 1.0
yearfractionFL = 1.0/2.0
floatingrate = floatrate[0]/100.0
pricing.WREpricingIRS( datesFL,  actualizationratesFL,   principalFL,  yearfractionFL,
                       ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, 
                       floatingrate, fixedrate, buyer, fixedleg, floatingleg, price, info)

# ( datesFL, actualizationratesFL, principalFL, yearfractionFL, ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, currentfloatingrate, fixedrate, buyer, fixedleg, floatingleg, price, info

 principal actualized from fixed leg=  0.90753785519466001     
   1.0000000000000000        2.1560000000000003E-002   4.5000000000000000     
 fixedleg=   1.0410159944002311     
 principal actualized from floating leg=  0.98690154134574448     
 test   1.0000000000000000        2.6370000000000001E-002  0.50000000000000000                1
 floatingleg=  0.99991383816838808     
 fixedleg=  0.13347813920557106     
 floatingleg=   9.2375982973728066E-002
 P=   4.1102156231842990E-002


In [32]:
quote=floatingleg[0]*100/fixedleg[0]
quote

np.float64(69.20682556973532)

In [33]:
datesFL

np.float64(0.5)

In [34]:
actualizationratesFL

np.float64(0.02637)

In [35]:
principalFL


1.0

In [36]:
yearfractionFL

0.5

In [37]:
floatingrate

np.float64(0.02637)

In [38]:
FLEG = (principalFL+floatingrate)*np.exp(-actualizationratesFL*datesFL)

In [39]:
FLEG-0.89484478529357070     

np.float64(0.11808134969746098)

In [40]:
actualizationratesFL

np.float64(0.02637)

In [41]:
actualizationratesFL = actratflot/100.0
actualizationratesFL

array([0.02637 , 0.02375 , 0.02278 , 0.02181 , 0.021675, 0.02154 ,
       0.02155 , 0.02156 , 0.02156 ])

In [42]:
actratflot

array([2.637 , 2.375 , 2.278 , 2.181 , 2.1675, 2.154 , 2.155 , 2.156 ,
       2.156 ])

# SW20251113_SOFR O/4.2587

In [43]:
#  SW20251113_SOFR O/4.2587  matu : 2025-11-13
#19/12/2024
# floating, ANNU
# fixe ANNU : 2025
# 1 PAIEMENT
#24/1/2025  ---> 
#24/1/2026
# quotation 100.016363

ibordates= np.array([1.,2.,3.,4.,5.,6.,7.,8.,9.,10.,15.,20.,30])
iborvaleurs= np.array([4.418842,4.588199,4.383796,4.442341,4.472094,4.487725,
                       4.603022,4.49304931,4.55046352,4.56296427,4.817077,4.969355, 4.76199173])

# pate fixe&flottante, actu :
m=1
deca=225
year=252.0
# t1=np.array([0,year,2*year, 3*year, 4*year])/252.0
t1=np.array([0])/252.0
t2=np.array([deca])/year
f12=np.zeros(m)
info = np.zeros(1,dtype='int64')
actrat=np.zeros(m)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, ibordates, iborvaleurs, m, t1, t2, interpmethod, actrat, info )

 r1=   4.4188419999999997     
 r2=   4.4188419999999997     
 f12=   4.4188419999999997     


In [44]:
ibordates

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 15., 20., 30.])

In [45]:
iborvaleurs

array([4.418842  , 4.588199  , 4.383796  , 4.442341  , 4.472094  ,
       4.487725  , 4.603022  , 4.49304931, 4.55046352, 4.56296427,
       4.817077  , 4.969355  , 4.76199173])

In [46]:
# pate flottante floatrate :
deca=126.0
semi=126.0
t1fl=np.array([0,0+semi,0+2*semi, 0+3*semi, 0+4*semi, 0+5*semi, 0+6*semi, 0+7*semi, 0+8*semi])/252.0
t2fl=np.array([deca,deca+semi,deca+2*semi, deca+3*semi, deca+4*semi, deca+5*semi, deca+6*semi, deca+7*semi, deca+8*semi])/year
info = np.zeros(1,dtype='int64')
floatrate=np.zeros(mfl)
interpmethod = 0
pricing.WREpricingCurve2Forward( ncurve, ibordates, iborvaleurs, mfl, t1fl, t2fl, interpmethod, floatrate, info )

 r1=   4.4188419999999997        4.4188419999999997        4.4188419999999997        4.5035205000000005        4.5881990000000004        4.4859974999999999        4.3837960000000002        4.4130684999999996        4.4423409999999999     
 r2=   4.4188419999999997        4.4188419999999997        4.5035205000000005        4.5881990000000004        4.4859974999999999        4.3837960000000002        4.4130684999999996        4.4423409999999999        4.4572175000000005     
 f12=   4.4188419999999997        4.4188419999999997        4.6728775000000020        4.8422345000000000        4.0771914999999979        3.8727885000000022        4.5887034999999941        4.6472485000000034        4.5762295000000037     


In [47]:
# jambe flottante
ndatesFL = m
datesFL = t2
actualizationratesFL = actrat/100.0
principalFL = 1.0
yearfractionFL = 1.0
floatingrate = floatrate/100.0

# jambe fixe
ndatesFIX = m
datesFIX = t2
actualizationratesFIX = actrat/100.0
principalFIX = 1.0
yearfractionFIX = 1.0
fixedrate = 4.2587/100.0
buyer=1

#valo
fixedleg = np.zeros(1)
floatingleg = np.zeros(1)
price = np.zeros(1)
pricing.WREpricingIRS2( ndatesFL,  datesFL,  actualizationratesFL,   principalFL,  yearfractionFL, floatingrate,
                      ndatesFIX, datesFIX, actualizationratesFIX, principalFIX, yearfractionFIX, 
                      fixedrate, buyer, fixedleg, floatingleg, price, info)

 principal actualized from fixed leg=  0.96131422492449592     
   1.0000000000000000        4.4188419999999999E-002  0.89285714285714290     
 fixedleg=   4.0939488896859506E-002
 principal actualized from floating leg=  0.96131422492449592     
 test   1.0000000000000000        4.4188419999999999E-002  0.89285714285714290                1
 floatingleg=   4.2478956722938097E-002
 fixedleg=   4.0939488896859506E-002
 floatingleg=   4.2478956722938097E-002
 P=  -1.5394678260785902E-003


In [48]:
ratio = 100*floatingleg/fixedleg 
ratio

array([103.7603494])

In [49]:
#  quantlib

# Let the today date whenwe want to value a instrument be
today = ql.Date(15,12,2024)

# we can set evaluationDate in QL as
ql.Settings.instance().evaluationDate = today
print(ql.Settings.instance().evaluationDate);


tenor = ql.Period('5y')
index = ql.Euribor6M()
fixedRate = 0.05
forwardStart = ql.Period("2D")

swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart)
swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart, swapType=ql.VanillaSwap.Payer)

December 15th, 2024


In [50]:
tenor

Period("5Y")

In [51]:
swap

<QuantLib.QuantLib.VanillaSwap; proxy of <Swig Object of type 'ext::shared_ptr< VanillaSwap > *' at 0x7fd54405c480> >

In [52]:
yts = ql.YieldTermStructureHandle(ql.FlatForward(2, ql.TARGET(), 0.5, ql.Actual360()))
engine = ql.DiscountingSwapEngine(yts)


In [53]:
yts

<QuantLib.QuantLib.YieldTermStructureHandle; proxy of <Swig Object of type 'Handle< YieldTermStructure > *' at 0x7fd525d586f0> >

In [54]:
engine

<QuantLib.QuantLib.DiscountingSwapEngine; proxy of <Swig Object of type 'ext::shared_ptr< DiscountingSwapEngine > *' at 0x7fd5683768b0> >

In [55]:
# https://quantlib-python-docs.readthedocs.io/en/latest/examples/fixedincome/vanillaswap.html

In [56]:
yts = ql.RelinkableYieldTermStructureHandle()

instruments = [
    ('depo', '6M', 0.025),
    ('swap', '1Y', 0.031),
    ('swap', '2Y', 0.032),
    ('swap', '3Y', 0.035)
]

helpers = ql.RateHelperVector()
index = ql.Euribor6M(yts)
for instrument, tenor, rate in instruments:
    if instrument == 'depo':
        helpers.append( ql.DepositRateHelper(rate, index) )
    if instrument == 'fra':
        monthsToStart = ql.Period(tenor).length()
        helpers.append( ql.FraRateHelper(rate, monthsToStart, index) )
    if instrument == 'swap':
        swapIndex = ql.EuriborSwapIsdaFixA(ql.Period(tenor))
        helpers.append( ql.SwapRateHelper(rate, swapIndex))
curve = ql.PiecewiseLogCubicDiscount(2, ql.TARGET(), helpers, ql.Actual365Fixed())

In [57]:
helpers


<QuantLib.QuantLib.RateHelperVector; proxy of <Swig Object of type 'std::vector< ext::shared_ptr< RateHelper > > *' at 0x7fd525d3e370> >

In [58]:
yts.linkTo(curve)
engine = ql.DiscountingSwapEngine(yts)

In [59]:
# Create a relinkable yield term structure handle and build a curve
tenor = ql.Period('2y')
fixedRate = 0.05
forwardStart = ql.Period("2D")

swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart, nominal=10e6, pricingEngine=engine)

In [60]:
# Link the built curve to the relinkable yield term structure handle and build a swap pricing engine
fairRate = swap.fairRate()
npv = swap.NPV()

print(f"Fair swap rate: {fairRate:.3%}")
print(f"Swap NPV: {npv:,.3f}")

Fair swap rate: 3.204%
Swap NPV: -343,146.359


In [61]:
# Build a vanilla swap and provide a pricing engine
tenor = ql.Period('2y')
fixedRate = 0.05
forwardStart = ql.Period("2D")

swap = ql.MakeVanillaSwap(tenor, index, fixedRate, forwardStart, nominal=10e6, pricingEngine=engine)


In [62]:
# Get the fair rate and NPV
fairRate = swap.fairRate()
npv = swap.NPV()

print(f"Fair swap rate: {fairRate:.3%}")
print(f"Swap NPV: {npv:,.3f}")




Fair swap rate: 3.204%
Swap NPV: -343,146.359


In [63]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

cashflows = pd.DataFrame({
    'date': cf.date(),
    'amount': cf.amount()
    } for cf in swap.leg(1))
display(cashflows)

,date,amount
0,"June 20th, 2025","126,901.87"
1,"December 22nd, 2025","183,557.18"
2,"June 22nd, 2026","163,989.32"
3,"December 21st, 2026","162,602.37"


In [64]:
cashflows = pd.DataFrame({
    'nominal': cf.nominal(),
    'accrualStartDate': cf.accrualStartDate().ISO(),
    'accrualEndDate': cf.accrualEndDate().ISO(),
    'rate': cf.rate(),
    'amount': cf.amount()
    } for cf in map(ql.as_coupon, swap.leg(1)))
display(cashflows)

,nominal,accrualStartDate,accrualEndDate,rate,amount
0,"10,000,000.00",2024-12-20,2025-06-20,0.03,"126,901.87"
1,"10,000,000.00",2025-06-20,2025-12-22,0.04,"183,557.18"
2,"10,000,000.00",2025-12-22,2026-06-22,0.03,"163,989.32"
3,"10,000,000.00",2026-06-22,2026-12-21,0.03,"162,602.37"
